In [1]:
import pandas as pd
import csv
import os
import requests
import urllib3
import csv
import time
import re
import numpy as np
# from random import randint
# from pandas import json_normalize
import censusgeocode as cg

In [2]:
os.chdir('C:/Users/jhuo/Documents/20220421_WABE_SB202/')

In [3]:
filename = '2022429_Georgia_Daily_VoterBase/Georgia_Daily_VoterBase.txt'

In [4]:
voter = pd.read_csv(filename, low_memory = False, error_bad_lines=False, sep = '|')

b'Skipping line 4381787: expected 63 fields, saw 114\n'


In [7]:
voter.RACE_DESC.value_counts()

White not of Hispanic Origin         3942041
Black not of Hispanic Origin         2231619
Unknown                               709314
Hispanic                              284592
Asian or Pacific Islander             201779
Other                                 147733
American Indian or Alaskan Native      28235
Name: RACE_DESC, dtype: int64

In [8]:
voter.RACE.value_counts()

WH    3942041
BH    2231619
U      709314
HP     284592
AP     201779
OT     147733
AI      28235
Name: RACE, dtype: int64

In [5]:
# cleaning up addresses
voter['RESIDENCE_STATE'] = 'GA'
voter['ZIP5'] =  [x[:5] if pd.isna(x) == False else x for x in voter.RESIDENCE_ZIPCODE ]
voter = voter.fillna('')
# strip whitespaces
cols = voter.select_dtypes(object).columns
voter[cols] = voter[cols].apply(lambda x: x.str.strip())

voter['st_address'] = [x + ' ' + y for (x,y) in zip(voter.RESIDENCE_HOUSE_NUMBER , voter.RESIDENCE_STREET_NAME)]

In [6]:
addresses = voter.iloc[:,[65,12,63,64]].copy().drop_duplicates()

In [7]:
addresses['onefield'] = [x + ', ' + y + ', ' + z + ', ' + a 
                         for (x,y,z,a) in 
                         zip(addresses.st_address,
                             addresses.RESIDENCE_CITY,
                             addresses.RESIDENCE_STATE,
                             addresses.ZIP5)]

In [8]:
addresses.head()

,st_address,RESIDENCE_CITY,RESIDENCE_STATE,ZIP5,onefield
0,237 MITCHELL RD,MAYSVILLE,GA,30558,"237 MITCHELL RD, MAYSVILLE, GA, 30558"
1,467 HICKORY CREEK RD,MAYSVILLE,GA,30558,"467 HICKORY CREEK RD, MAYSVILLE, GA, 30558"
2,733 CARSON SEGARS RD,MAYSVILLE,GA,30558,"733 CARSON SEGARS RD, MAYSVILLE, GA, 30558"
3,11 W RIDGEWAY RD,MAYSVILLE,GA,30558,"11 W RIDGEWAY RD, MAYSVILLE, GA, 30558"
4,262 HIGHWAY 52,MAYSVILLE,GA,30558,"262 HIGHWAY 52, MAYSVILLE, GA, 30558"


In [9]:
# checking out how many of these addresses have already been geocoded
res = ['geocode_results/' + x for x in os.listdir(os.getcwd() + '/geocode_results/')]

In [10]:
masterresults = pd.DataFrame()
censusresults = pd.DataFrame()
censusapiresults = pd.DataFrame()

In [11]:
# parse files according to different geocoders
for file in res:
    # R and python outputs files have headers; inherit them
    if (('GeoCode' in file) | ('pythongeocode' in file)):
        df = pd.read_csv(file)
        masterresults = masterresults.append(df)
    # census website outputs do not have headers; add them
    if 'GeocodeResults' in file:
        try:
            df = pd.read_csv(file, names =  ['id', 'address','match','matchtype','parsed','coordinate','tigerlineid','side'])
            censusresults = censusresults.append(df)
        except Exception:
            print (file)
            continue
    else:
        df = pd.read_csv(file)
        censusapiresults = censusapiresults.append(df)

since the addresses in the voter files were really clean, the idea is to parse the fields into a one line address that can be joined to all the results without worrying about the various ids

In [12]:
masterresults1 = masterresults.copy()

In [13]:
masterresults1['zip5'] = ['' if pd.isna(x) == True else str(int(x)) for x in masterresults1['zip'] ]

masterresults1.address = [b if pd.isna(b) == False else str(x) + ', ' + str(y) + ', ' + str(z) + ', ' + str(a) 
                         for (x,y,z,a,b) in 
                         zip(masterresults1.street,
                             masterresults1.city,
                             masterresults1.state,
                             masterresults1.zip5,
                            masterresults1.address)]

In [14]:
masterresults1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2664245 entries, 0 to 9904
Data columns (total 16 columns):
 #   Column       Dtype  
---  ------       -----  
 0   Unnamed: 0   int64  
 1   id           float64
 2   street       object 
 3   city         object 
 4   state        object 
 5   zip          float64
 6   lat          float64
 7   long         float64
 8   address      object 
 9   match        object 
 10  matchtype    object 
 11  parsed       object 
 12  tigerlineid  float64
 13  side         object 
 14  lon          float64
 15  zip5         object 
dtypes: float64(6), int64(1), object(9)
memory usage: 345.6+ MB


In [15]:
masterresults1.long = [x if pd.isna(x) == False else y for (x,y) in zip(masterresults1.long, masterresults1.lon)]

In [16]:
masterresults1ft = masterresults1[['address','lat','long']].copy()

Parsing Census website batch results

In [17]:
censusresults.head()

,id,address,match,matchtype,parsed,coordinate,tigerlineid,side
0,133265,"1038 CADDENWOODS DR, AUGUSTA, GA, 30906",Match,Exact,"1038 CADDENWOODS DR, AUGUSTA, GA, 30906","-82.04344,33.396816",617769247.0,L
1,134596,"156 WINCHESTER DR, LEESBURG, GA, 31763",Match,Exact,"156 WINCHESTER DR, LEESBURG, GA, 31763","-84.15729,31.630701",647510725.0,R
2,133266,"606 PAULINE JENKINS ST, WAYNESBORO, GA, 30830",No_Match,NaN,NaN,NaN,NaN,NaN
3,134597,"2826 APRIL DR, AUGUSTA, GA, 30906",Match,Exact,"2826 APRIL DR, AUGUSTA, GA, 30906","-82.09449,33.426952",55634892.0,R
4,134598,"829 BURLINGTON DR, AUGUSTA, GA, 30909",Match,Exact,"829 BURLINGTON DR, AUGUSTA, GA, 30909","-82.10492,33.449215",651001177.0,R


In [18]:
censusresults['split'] = [x.split(',') if pd.isna(x) == False else [np.nan,np.nan] for x in censusresults['coordinate']]
censusresults['lat'] = [x[1] for x in censusresults.split]
censusresults['long'] = [x[0] for x in censusresults.split]

In [19]:
censusresultsft = censusresults[['parsed','lat','long']].copy()
censusresultsft.columns = ['address','lat','long']

In [20]:
censusresultsft[pd.isna(censusresultsft.lat) == True].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66577 entries, 2 to 9994
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   address  0 non-null      object
 1   lat      0 non-null      object
 2   long     0 non-null      object
dtypes: object(3)
memory usage: 2.0+ MB


In [21]:
censusresultsft[pd.isna(censusresultsft.lat) == False].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 523258 entries, 0 to 9997
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   address  523258 non-null  object
 1   lat      523258 non-null  object
 2   long     523258 non-null  object
dtypes: object(3)
memory usage: 16.0+ MB


Parsing census api results

In [22]:
censusapiresults1 = censusapiresults.copy()

In [23]:
censusapiresults1.head()

,Unnamed: 0,addressComponents,coordinates,input,matchedAddress,tigerLine,url,id,street,city,...,zip,lat,long,address,match,matchtype,parsed,tigerlineid,side,lon
0,0,"{'fromAddress': '399', 'toAddress': '149', 'pr...","{'x': -82.50677, 'y': 33.46766}",305 W HALL ST THOMSON GA 30824,"305 W HALL ST, THOMSON, GA, 30824","{'tigerLineId': '43141641', 'side': 'R'}",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"{'fromAddress': '999', 'toAddress': '1', 'preQ...","{'x': -82.74473, 'y': 33.000156}",239 ARCHER RD SANDERSVILLE GA 31082,"239 ARCHER RD, SANDERSVILLE, GA, 31082","{'tigerLineId': '16761454', 'side': 'R'}",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"{'fromAddress': '2301', 'toAddress': '2599', '...","{'x': -82.47879, 'y': 33.41434}",2445 OLD MILLEDGEVILLE RD THOMSON GA 30824,"2445 OLD MILLEDGEVILLE RD, THOMSON, GA, 30824","{'tigerLineId': '43150320', 'side': 'L'}",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"{'fromAddress': '3662', 'toAddress': '3016', '...","{'x': -82.445885, 'y': 33.41531}",3244 WIRE RD DEARING GA 30808,"3244 WIRE RD, DEARING, GA, 30808","{'tigerLineId': '43147978', 'side': 'L'}",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,"{'fromAddress': '1000', 'toAddress': '1098', '...","{'x': -82.544075, 'y': 33.493362}",1064 TANYARD CREEK DR THOMSON GA 30824,"1064 TANYARD CREEK DR, THOMSON, GA, 30824","{'tigerLineId': '43157559', 'side': 'L'}",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
censusapiresults1[pd.isna(censusapiresults1.coordinates) == True].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2669111 entries, 48 to 9904
Data columns (total 21 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Unnamed: 0         int64  
 1   addressComponents  object 
 2   coordinates        object 
 3   input              object 
 4   matchedAddress     object 
 5   tigerLine          object 
 6   url                object 
 7   id                 float64
 8   street             object 
 9   city               object 
 10  state              object 
 11  zip                float64
 12  lat                float64
 13  long               float64
 14  address            object 
 15  match              object 
 16  matchtype          object 
 17  parsed             object 
 18  tigerlineid        float64
 19  side               object 
 20  lon                float64
dtypes: float64(6), int64(1), object(14)
memory usage: 448.0+ MB


In [25]:
censusapiresults1['lat'] = [x.split(':')[2].split('}')[0] if pd.isna(x) == False else '' for x in censusapiresults1.coordinates]
censusapiresults1['long'] = [x.split(':')[1].split(',')[0] if pd.isna(x) == False else '' for x in censusapiresults1.coordinates]
censusapiresults1.lat = censusapiresults1.lat.str.strip()
censusapiresults1.long = censusapiresults1.long.str.strip()
censusapiresults1 = censusapiresults1.drop('coordinates', axis = 1)

In [26]:
censusapiresults1ft = censusapiresults1[['matchedAddress','lat','long']].copy()
censusapiresults1ft.columns = ['address','lat','long']

In [27]:
censusapiresults1ft[censusapiresults1ft.lat== ''].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2669111 entries, 48 to 9904
Data columns (total 3 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   address  object
 1   lat      object
 2   long     object
dtypes: object(3)
memory usage: 81.5+ MB


In [28]:
censusapiresults1ft = censusapiresults1ft.replace(r'^\s*$', np.nan, regex=True)

In [29]:
censusapiresults1ft[pd.isna(censusapiresults1ft.lat) == True].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2669111 entries, 48 to 9904
Data columns (total 3 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   address  object
 1   lat      object
 2   long     object
dtypes: object(3)
memory usage: 81.5+ MB


In [30]:
allcensusresults = pd.concat([censusapiresults1ft,censusresultsft,masterresults1ft]).drop_duplicates()

In [31]:
allcensusresults1 = allcensusresults.copy()

In [32]:
allcensusresults1.lat = allcensusresults1.lat.astype('float')
allcensusresults1.long = allcensusresults1.long.astype('float')

In [33]:
# round to 6 decimals 
allcensusresults1.lat = [round(x,6) for x in allcensusresults1.lat]
allcensusresults1.long = [round(x,6) for x in allcensusresults1.long]

In [34]:
allcensusresults1 = allcensusresults1.drop_duplicates()

In [35]:
# naildown one lat long coordinate for each address. A possible reason is difference benchmarking.
allcensusresultscnt = pd.DataFrame(allcensusresults1.groupby('address')['lat'].max())

In [36]:
allcensusresultscnt['keep'] = True

In [37]:
allcensusresultscnt = allcensusresultscnt.reset_index()

In [38]:
allcensusresults2 = pd.merge(allcensusresults1, allcensusresultscnt, on = ['address','lat'], how = 'left')

In [39]:
allcensusresults3 = allcensusresults2[allcensusresults2.keep == True].copy()

In [40]:
allcensusresults3 = allcensusresults3.drop('keep', axis = 1)

merging all geocoding results with the new voter file

In [41]:
addressesmerge = pd.merge(addresses, allcensusresults3, left_on = 'onefield', right_on = 'address', how = 'outer')

In [42]:
addressesmerge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3407559 entries, 0 to 3407558
Data columns (total 8 columns):
 #   Column           Dtype  
---  ------           -----  
 0   st_address       object 
 1   RESIDENCE_CITY   object 
 2   RESIDENCE_STATE  object 
 3   ZIP5             object 
 4   onefield         object 
 5   address          object 
 6   lat              float64
 7   long             float64
dtypes: float64(2), object(6)
memory usage: 234.0+ MB


In [43]:
# over three million matched.
addressesmerge[(pd.isna(addressesmerge.onefield) == False) & (pd.isna(addressesmerge.address) == False)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3059343 entries, 0 to 3201659
Data columns (total 8 columns):
 #   Column           Dtype  
---  ------           -----  
 0   st_address       object 
 1   RESIDENCE_CITY   object 
 2   RESIDENCE_STATE  object 
 3   ZIP5             object 
 4   onefield         object 
 5   address          object 
 6   lat              float64
 7   long             float64
dtypes: float64(2), object(6)
memory usage: 210.1+ MB


In [44]:
# 142317 in the new voter file don't have a result. looking into to see if any are due to join failures
addressesmerge[(pd.isna(addressesmerge.onefield) == False) & (pd.isna(addressesmerge.address) == True)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142317 entries, 6 to 3201574
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   st_address       142317 non-null  object 
 1   RESIDENCE_CITY   142317 non-null  object 
 2   RESIDENCE_STATE  142317 non-null  object 
 3   ZIP5             142317 non-null  object 
 4   onefield         142317 non-null  object 
 5   address          0 non-null       object 
 6   lat              0 non-null       float64
 7   long             0 non-null       float64
dtypes: float64(2), object(6)
memory usage: 9.8+ MB


In [45]:
# that's pretty impressive match rate actually
len(addressesmerge[(pd.isna(addressesmerge.onefield) == False) & (pd.isna(addressesmerge.address) == True)])/len(addressesmerge[(pd.isna(addressesmerge.onefield) == False)])

0.044451003541912636

In [46]:
# about 2,000 is "missing address"
addressesmerge[ (pd.isna(addressesmerge.onefield) == False) & (addressesmerge.onefield.str.contains('missing addr',case = False)) ].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1859 entries, 3326 to 3196435
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   st_address       1859 non-null   object 
 1   RESIDENCE_CITY   1859 non-null   object 
 2   RESIDENCE_STATE  1859 non-null   object 
 3   ZIP5             1859 non-null   object 
 4   onefield         1859 non-null   object 
 5   address          517 non-null    object 
 6   lat              0 non-null      float64
 7   long             0 non-null      float64
dtypes: float64(2), object(6)
memory usage: 130.7+ KB


In [47]:
addressesmerge[(pd.isna(addressesmerge.onefield) == False)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3201660 entries, 0 to 3201659
Data columns (total 8 columns):
 #   Column           Dtype  
---  ------           -----  
 0   st_address       object 
 1   RESIDENCE_CITY   object 
 2   RESIDENCE_STATE  object 
 3   ZIP5             object 
 4   onefield         object 
 5   address          object 
 6   lat              float64
 7   long             float64
dtypes: float64(2), object(6)
memory usage: 219.8+ MB


In [48]:
# THIS SUBSET ISN'T WORKING BUT NVM
addressesmerge[ ~((addressesmerge.onefield.str.contains('MISSING ADDR', case = False)) | (pd.isna(addressesmerge.onefield) == True))].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3405700 entries, 0 to 3407558
Data columns (total 8 columns):
 #   Column           Dtype  
---  ------           -----  
 0   st_address       object 
 1   RESIDENCE_CITY   object 
 2   RESIDENCE_STATE  object 
 3   ZIP5             object 
 4   onefield         object 
 5   address          object 
 6   lat              float64
 7   long             float64
dtypes: float64(2), object(6)
memory usage: 233.9+ MB


In [49]:
addressesmerge[(pd.isna(addressesmerge.onefield) == False) & (pd.isna(addressesmerge.address) == True)][['st_address','RESIDENCE_CITY','RESIDENCE_STATE','ZIP5']].iloc[:2490,:].to_csv('20220503_tamu_geocoding_batch1.csv')

In [50]:
addressesmerge[(pd.isna(addressesmerge.onefield) == False) & (pd.isna(addressesmerge.address) == True)][['st_address','RESIDENCE_CITY','RESIDENCE_STATE','ZIP5']].iloc[:2490,:].to_csv('20220503_tamu_geocoding_batch1.csv')